In [43]:
import pandas as pd

# Load the CSV files
rentals_df = pd.read_csv('data/half-moon-bay/half-moon-bay-rentals.csv')
listings_df = pd.read_csv('data/half-moon-bay/half-moon-bay-listings.csv')

# Display the first few rows of each DataFrame
# print("Rentals DataFrame:")
print(rentals_df.head())

# print("\nListings DataFrame:")
# print(listings_df.head())

                                     id  \
0  6c40b677-d423-404c-bd9f-bb151d85b5a0   
1  6c40b677-d423-404c-bd9f-bb151d85b5a0   
2  c3c20ee8-87aa-487d-82a3-f7d0397f344b   
3  3ff9df86-3817-4d69-afe7-1153be36e740   
4  c3c20ee8-87aa-487d-82a3-f7d0397f344b   

                                   record_range accommodation_type  bathrooms  \
0  listing-553a1e58-a13c-4bef-a2e9-de9df8419c05                NaN        NaN   
1  listing-3f690852-2757-45a7-aef3-5d10318b45b5                NaN        NaN   
2  listing-5ca7e4c3-83e2-4c22-afc7-ae2e82094b1f                NaN        NaN   
3  listing-082867cd-3854-421e-99a0-547e3d818cef                NaN        3.0   
4  listing-c7d8e38c-85b5-4aae-a458-0dc1a0e70a31                NaN        NaN   

   bedrooms  beds                  created_at         created_by_email  \
0       NaN   NaN  2024-10-21 15:11:19.617508  bearlakeheidi@gmail.com   
1       NaN   NaN  2024-10-21 15:11:19.617508  bearlakeheidi@gmail.com   
2       NaN   NaN  2024-10-21 15

In [44]:
rentals_subset = rentals_df[['listing_id', 'id']]
listings_subset = listings_df[['ID', 'Link']]
# print(listings_subset.head())
# print(rentals_subset.head())

In [45]:
merged_df = pd.merge(listings_subset, rentals_subset, left_on='ID', right_on='listing_id')
# print(merged_df.head())

In [46]:
merged_df.rename(columns={
    'ID': 'listing_table_id',
    'Link': 'listing_table_link',
    'listing_id': 'rental_table_listing_id',
    'id': 'rental_table_id'
}, inplace=True)

# print(merged_df.head())

In [47]:
new_df = merged_df[['rental_table_id', 'listing_table_link']]
# print(new_df.head(10))

In [48]:
# Create a new DataFrame with unique rental_table_id
unique_rentals = new_df.drop_duplicates(subset=['rental_table_id'])

# Create a dictionary to hold the listing URLs for each rental_table_id
rental_urls = {}

for _, row in new_df.iterrows():
    rental_id = row['rental_table_id']
    link = row['listing_table_link']
    if rental_id not in rental_urls:
        rental_urls[rental_id] = []
    rental_urls[rental_id].append(link)

# Add the listing URLs to the unique_rentals DataFrame
unique_rentals['listing_url_1'] = unique_rentals['rental_table_id'].apply(lambda x: rental_urls[x][0] if len(rental_urls[x]) > 0 else None)
unique_rentals['listing_url_2'] = unique_rentals['rental_table_id'].apply(lambda x: rental_urls[x][1] if len(rental_urls[x]) > 1 else None)
unique_rentals['listing_url_3'] = unique_rentals['rental_table_id'].apply(lambda x: rental_urls[x][2] if len(rental_urls[x]) > 2 else None)
# Remove the listing_table_link column
unique_rentals.drop(columns=['listing_table_link'], inplace=True)
# Rename the column 'rental_table_id' to 'rental_id'
unique_rentals.rename(columns={'rental_table_id': 'rental_id'}, inplace=True)


C:\Users\k_jac\AppData\Local\Temp\ipykernel_12176\1435217596.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_rentals['listing_url_1'] = unique_rentals['rental_table_id'].apply(lambda x: rental_urls[x][0] if len(rental_urls[x]) > 0 else None)
C:\Users\k_jac\AppData\Local\Temp\ipykernel_12176\1435217596.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_rentals['listing_url_2'] = unique_rentals['rental_table_id'].apply(lambda x: rental_urls[x][1] if len(rental_urls[x]) > 1 else None)
C:\U

In [49]:
unique_rentals.sort_values(by='rental_id', inplace=True)
# print(unique_rentals.head(10))

C:\Users\k_jac\AppData\Local\Temp\ipykernel_12176\2461339577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_rentals.sort_values(by='rental_id', inplace=True)


In [51]:
unique_rentals.to_csv('data/half-moon-bay/flattened.csv', index=False)